* 2021년 시도별 평균 미세먼지 농도에 차이가 있는가. --> ANOVA
* 2011 ~ 2021년 봄(3,4,5월)과 겨울(10,11,12월) 평균 미세먼지 농도에 차이가 있는가. --> ttest_ind

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

data_file = '미세먼지_PM10__월별_도시별_대기오염도_20230322213856.csv'
np_data = pd.read_csv(data_file, encoding = 'cp949').to_numpy()

col_names = np_data[0]
row_index = np_data[1:, 0]
np_data = np_data[1:, 1:]

#flatten <--- 카피본 생성
#ravel  <---- 원본 사용

np_data_flat = np_data.flatten()
#print(np_data_flat, np_data_flat.shape)

for i, value in enumerate (np_data_flat):
    if '*' in value:
        star_index = value.find('*')
        np_data_flat[i] = value[:star_index]
        
np_data_flat_re = np_data_flat.reshape(132, 4)
np_data = np_data_flat_re.astype(np.int64)

print(np_data)

#print(col_names)
#print(row_index)


[[44 38 39 29]
 [75 57 63 52]
 [65 58 59 48]
 [56 56 52 50]
 [72 76 67 76]
 [44 52 43 43]
 [28 41 28 28]
 [27 35 25 29]
 [29 33 28 26]
 [42 43 42 40]
 [38 41 40 36]
 [46 41 43 36]
 [60 45 51 47]
 [50 46 45 39]
 [46 47 46 45]
 [51 53 49 41]
 [52 58 53 51]
 [40 38 37 23]
 [28 40 27 21]
 [22 34 19 19]
 [27 35 28 31]
 [33 42 33 37]
 [42 47 44 44]
 [41 36 38 26]
 [64 56 58 44]
 [45 47 47 37]
 [55 66 54 56]
 [52 49 48 46]
 [55 62 53 58]
 [40 47 40 36]
 [34 47 28 30]
 [35 47 28 31]
 [28 33 29 34]
 [29 38 30 39]
 [42 43 39 50]
 [55 47 46 45]
 [57 57 56 56]
 [57 45 51 48]
 [60 50 50 57]
 [58 53 45 51]
 [63 71 59 81]
 [38 54 39 54]
 [38 48 31 41]
 [29 36 23 35]
 [29 39 27 34]
 [33 39 29 38]
 [45 47 39 40]
 [44 42 43 49]
 [49 47 52 51]
 [84 63 68 63]
 [71 56 68 62]
 [45 48 40 48]
 [45 56 49 42]
 [35 49 40 41]
 [30 37 31 36]
 [34 39 32 39]
 [28 29 27 33]
 [44 43 50 51]
 [33 33 37 41]
 [48 41 49 42]
 [50 45 48 45]
 [45 46 46 53]
 [64 52 59 53]
 [71 60 64 61]
 [56 53 49 56]
 [46 39 38 38]
 [33 29 27

In [19]:
# 다른 방법(2차원 배열로 처리)

#test_str_a = '123*'
#test_str_b = '123**'

#print('*' in test_str_a)
#print(test_str_a.find('*'))
#print(test_str_a[:3])

row_count, col_count = np_data.shape   # tuple로 반환을 받는다.
print(row_count, col_count)

for row_index in range(row_count):
    for col_index in range(col_count):
        value_str = np_data[row_index, col_index]
        if '*' in value_str:
            star_index = value_str.find('*')
            np_data[row_index, col_index] = value_str[:star_index]
            
np_data = np_data.astype(np.int64)   

In [20]:
print(col_names)
print(row_index)
print(np_data)

['시점' '서울특별시' '부산광역시' '대전광역시' '제주특별자치도']
['2011.01' '2011.02' '2011.03' '2011.04' '2011.05' '2011.06' '2011.07'
 '2011.08' '2011.09' '2011.10' '2011.11' '2011.12' '2012.01' '2012.02'
 '2012.03' '2012.04' '2012.05' '2012.06' '2012.07' '2012.08' '2012.09'
 '2012.10' '2012.11' '2012.12' '2013.01' '2013.02' '2013.03' '2013.04'
 '2013.05' '2013.06' '2013.07' '2013.08' '2013.09' '2013.10' '2013.11'
 '2013.12' '2014.01' '2014.02' '2014.03' '2014.04' '2014.05' '2014.06'
 '2014.07' '2014.08' '2014.09' '2014.10' '2014.11' '2014.12' '2015.01'
 '2015.02' '2015.03' '2015.04' '2015.05' '2015.06' '2015.07' '2015.08'
 '2015.09' '2015.10' '2015.11' '2015.12' '2016.01' '2016.02' '2016.03'
 '2016.04' '2016.05' '2016.06' '2016.07' '2016.08' '2016.09' '2016.10'
 '2016.11' '2016.12' '2017.01' '2017.02' '2017.03' '2017.04' '2017.05'
 '2017.06' '2017.07' '2017.08' '2017.09' '2017.10' '2017.11' '2017.12'
 '2018.01' '2018.02' '2018.03' '2018.04' '2018.05' '2018.06' '2018.07'
 '2018.08' '2018.09' '2018.10' '2018

In [25]:
data_2021 = np_data[-12:]
print(data_2021, data_2021.shape)

mean_2021 = np.mean(data_2021, axis = 0)
print(mean_2021)
anova_result = stats.f_oneway(data_2021[:,0], data_2021[:,1], data_2021[:,2], data_2021[:,3])
print(anova_result)
# 귀무가설 : mean 값에 차이가 없음. (채택)
# 평균값에 차이가 없다고 봐도 무방하다. --> 2021년도 4개의 시도의 미세먼지 PM10 측정량 평균은 차이가 없다.

[[38 36 39 34]
 [48 37 42 36]
 [67 60 63 76]
 [42 37 41 37]
 [61 36 56 44]
 [33 26 31 27]
 [24 18 16 17]
 [22 19 18 16]
 [15 17 15 19]
 [27 23 27 24]
 [45 34 38 38]
 [39 31 40 30]] (12, 4)
[38.41666667 31.16666667 35.5        33.16666667]
F_onewayResult(statistic=0.5352888611262938, pvalue=0.6605335914872638)


In [43]:
#* 2011 ~ 2021년 봄(3,4,5월)과 겨울(10,11,12월) 평균 미세먼지 농도에 차이가 있는가. --> ttest_ind
#print(row_index)
#print(np_data)
a = row_index.astype(np.float64)
b = (a%1.0)*100 # 월만 가져옴

filter_spring = (b>2) & (b<5) # 인덱스 값
filter_winter = (b>9) & (b<12)
#print(filter_spring)
#print(filter_winter)

spring_data = np_data[filter_spring].flatten()
winter_data = np_data[filter_winter].flatten()
print(np.mean(spring_data), np.mean(winter_data)) 
print(stats.ttest_ind(spring_data, winter_data)) # 완전히 다른 값이고 봄에 미세먼지가 더 많다.

52.67424242424242 39.46969696969697
Ttest_indResult(statistic=12.232958045607596, pvalue=1.6003353734189516e-27)


In [44]:
# 2011 ~ 2021 년 까지, 각 도시와 연도별 미세먼지 농도가 가장 높았던 월은?
# ex. 2021년도 부산 __월이 미세먼지 농도가 가장 높았다.
for yr in range(2011, 2022):
    start = 12 * (yr-2011)
    stop = start+12
    yr_data = np_data[start: stop]
    print(np.argmax(yr_data, axis=0)+1)  # np.argmax == max value의 index를 달라.
    # +1을 해서 index를 바로 month 값으로 받았다.

[2 5 5 5]
[1 5 5 5]
[1 3 1 5]
[5 5 5 5]
[2 2 2 2]
[4 4 4 4]
[5 5 5 4]
[ 2  4 11  4]
[3 3 3 3]
[3 4 1 4]
[3 3 3 3]
